In [1]:
import pandas as pd
import os
import numpy as np
import plotly
import plotly.express as px
import SimpleITK as sitk
import sys
import torch

sys.path.append("/mnt/raid/C1_ML_Analysis/source/famli-ultra-sim/dl/")
from transforms.ultrasound_transforms import RealUSEvalTransformsV2
from nets import cut

from monai.transforms import (    
    LoadImage
)


In [9]:
# model_fn = '/mnt/raid/C1_ML_Analysis/train_output/Cut/butterfly2voluson/v0.2/epoch=56-val_loss=3.77.ckpt'

# model = cut.CutG.load_from_checkpoint(model_fn).eval()
# x = torch.rand(1, 1, 256, 256)
# model.to_torchscript(file_path="/mnt/famli_netapp_shared/C1_ML_Analysis/trained_models/cut/butterfly2voluson_v0.2_epoch=56-val_loss=3.77.pt", method="trace", example_inputs=x.cuda())

In [3]:
model = torch.jit.load("/mnt/famli_netapp_shared/C1_ML_Analysis/trained_models/cut/butterfly2voluson.pt")
model = model.cuda()

In [4]:
df = pd.read_parquet("/mnt/raid/C1_ML_Analysis/CSV_files/extract_frames_blind_sweeps_butterfly_test.parquet")

In [ ]:
df.columns

In [ ]:
fn = os.path.join('/mnt/raid/C1_ML_Analysis', df['img_path'].loc[np.random.randint(len(df))])
img = sitk.ReadImage(fn)
img_np = sitk.GetArrayFromImage(img)

# img_t = LoadImage()(fn)

# img_t = RealUSEvalTransformsV2()(img_np.astype(float))

# img_t.max()
px.imshow(img_np[:,:,0])

In [ ]:
img_t = torch.tensor(img_np[:,:,0]).unsqueeze(0).unsqueeze(0).float().cuda()
img_out = model(img_t)
px.imshow(img_out.cpu().detach().numpy().squeeze())